In [1]:
from astrapy.db import AstraDB
from langchain_astradb import AstraDBVectorStore
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
import json
import os
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

token = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
dbEndpoint = os.getenv("ASTRA_DB_API_ENDPOINT")
openaiToken = os.getenv("OPENAI_API_KEY")
csvPath = ("../project/data/rotten_tomatoes_movies.csv")
collectionName = "vector_movies"
cleanedCsvPath = ("../project/data/moviesDataSetCleaned.csv")

# Configurar el motor de OpenAI
engine = "gpt-4"
embeddings = OpenAIEmbeddings(api_key=openaiToken, model="text-embedding-3-large")

# def get_embeddings(text):
#     query_results = embeddings.embed_query(text)
#     print(query_results)
#     return query_results

In [2]:
# Clean the dataset before we try sending it to AstraDB
columns = ['movie_title', 'genres', 'directors', 'tomatometer_rating']
df = pd.read_csv(csvPath)
df = df[columns]
df.columns = [column.strip() for column in df.columns]
df.dropna(axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

# dfCombined = pd.DataFrame(columns=['movies'])
df['summary'] = df['movie_title'].astype(str) + ": " + df['directors'].astype(str) + ": " + df['tomatometer_rating'].astype(str)
# dfCombined.to_csv(cleanedCsvPath, index=False)

In [5]:
# Initialization
# db = AstraDB(
#   token=token,
#   api_endpoint=dbEndpoint)
# # Create a collection
# collection = db.create_collection(collectionName, dimension=3072, metric="cosine")

# Initialize Langchain vector store
vstore = AstraDBVectorStore (
    embedding=embeddings,
    collection_name=collectionName,
    token=token,
    api_endpoint=dbEndpoint
)
# print (f"Connected to AstraDB: {db.get_collections()}")

# Procesar cada línea y obtener los embeddings
# Leer el documento de texto
documents = []

for index, row in enumerate(df['summary']):
    metadata = {"genres": df['genres'][index]}
    # print(f"INDEX: {i} \n We are in row {row} and the genres are: \n {df['genres'][index]}")
    document = Document(page_content=row, metadata=metadata)
    documents.append(document)
    if len(documents) == 15:
        inserted_ids = vstore.add_documents(documents)
        print(f"\nInserted {len(inserted_ids)} documents.")
        documents = []
        inserted_ids = []
        print(f"im at index: {index}")
# with open(cleanedCsvPath, 'r', encoding="utf8") as file:
#     lines = file.readlines()
    
# if os.path.exists('documents.json'):
#     with open('documents.json', 'r') as file:
#         documents = json.load(file)
#     for document in documents:
#         res = collection.upsert_one(document)
# else:
#     for index, line in enumerate(lines):
#         # Extraer el texto de cada línea
#         # Obtener el embedding para el texto
#         vector = get_embeddings(line)
#         # Crear el documento
#         document = {
#             "_id": str(index + 1),
#             "text": line,
#             "vector": vector
#         }
#         # Insertar el documento en la base de datos
#         # Reemplaza 'collection' con tu objeto de colección de la base de datos 
#         # res = vstore.add_documents(document)  # Aqui usamos upsert si existe se actualiza si no, se crea # This errors out with 'str' object has no attribute 'page_content'
#         res = collection.upsert_one(document)
#         documents.append(document)
        
#         # Guardar los documentos en un archivo JSON
#         with open ('documents.json', 'w') as file:
#             json.dump(documents, file)
#         print (documents)


Inserted 15 documents.
im at index: 14

Inserted 15 documents.
im at index: 29

Inserted 15 documents.
im at index: 44

Inserted 15 documents.
im at index: 59

Inserted 15 documents.
im at index: 74


KeyboardInterrupt: 